In [220]:
from importlib import reload
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [277]:
# import modules I've written
try:
    reload(m)
    from src.models import *
except:
    import src.models as m
    from src.models import *
    print('reload failed')
try:
    reload(d)
    from src.dataMethods import *
except:
    import src.dataMethods as d
    from src.dataMethods import *
    print('reload failed')


In [267]:
# get data frame with variables of interest
df_gesture = CSV2VoI('data/recordings/gesturing_small.csv')
df_nogesture = CSV2VoI('data/recordings/no_gesture_small.csv')

mean fps: 115.25
skipping every 5 frames
Found left hand data:  True
Found right hand data:  True
mean fps: 115.22
skipping every 5 frames
Found left hand data:  True
Found right hand data:  True


(1210, 21)

In [276]:
# extract X and y
X1, _, = df2X_y(df_gesture)
X0, _ = df2X_y(df_nogesture)
X1.shape
X0.shape

       right_index_tipPosition_0  right_index_tipPosition_1  \
0                       9.311210                 135.513000   
5                       9.738785                 135.680695   
10                     10.301114                 135.258041   
15                     10.833519                 134.544495   
20                     11.213173                 134.505508   
25                     11.557329                 134.929718   
30                     11.660778                 135.842392   
35                     11.625224                 136.620590   
40                     12.070473                 137.474091   
45                     12.059304                 137.346115   
50                     12.540459                 138.827667   
55                     13.721355                 141.094421   
60                     16.266977                 141.246902   
65                     19.183661                 147.181274   
70                     20.238131                 148.68

(1210, 21)

In [278]:
n_frames = 30
# separate into examples 120 frames long each
X1 = np.array(split2examples(X1, 30))
X0 = np.array(split2examples(X0, 30))
X1.shape
X0.shape

(40, 30, 21)

In [279]:
# get y labels
y1 = np.array([1 for i in X1])
y0 = np.array([0 for i in X0])
print(y1[:2])
print(y0[:2])

[1 1]
[0 0]


In [280]:
X = np.concatenate((X1, X0))
y = np.concatenate((y1, y0))

# https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison for random shuffling like this
state = np.random.get_state()
np.random.shuffle(X)
np.random.set_state(state)
np.random.shuffle(y)

In [281]:
model = many2one_model(n_frames=n_frames, n_gestures=1, rnn_units=16)

Model: "many2one"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 30, 21)]          0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 32)                4864      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 33        
Total params: 4,897
Trainable params: 4,897
Non-trainable params: 0
_________________________________________________________________


In [285]:
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=100)

Train on 106 samples
Epoch 1/100
106/106 [==============================] - 3s 31ms/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 2/100
106/106 [==============================] - 0s 545us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 3/100
106/106 [==============================] - 0s 480us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 4/100
106/106 [==============================] - 0s 480us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 5/100
106/106 [==============================] - 0s 527us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 6/100
106/106 [==============================] - 0s 480us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 7/100
106/106 [==============================] - 0s 508us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 8/100
106/106 [==============================] - 0s 508us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 9/100
106/106 [==============================] - 0s 470us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 10/100
106/106 [=============

106/106 [==============================] - 0s 499us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 79/100
106/106 [==============================] - 0s 508us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 80/100
106/106 [==============================] - 0s 461us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 81/100
106/106 [==============================] - 0s 470us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 82/100
106/106 [==============================] - 0s 470us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 83/100
106/106 [==============================] - 0s 461us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 84/100
106/106 [==============================] - 0s 480us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 85/100
106/106 [==============================] - 0s 480us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 86/100
106/106 [==============================] - 0s 470us/sample - loss: 5.7861 - accuracy: 0.6226
Epoch 87/100
106/106 [==============================] - 0s 

In [274]:
pdf = pd.DataFrame()
pdf['a'] = [1,2,3]
pdf['b'] = [-3,0,3]
pdf['c'] = [0, 0.1, np.nan]
pdf.dropna()

,a,b,c
0,1,-3,0.0
1,2,0,0.1
